In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# File paths
dataset_paths = {
    "2018": "2018_cleaned_personal_property_list.csv",
    "2019": "2019_cleaned_personal_property_list.csv",
    "2020": "2020_cleaned_personal_property_list.csv",
    "2021": "2021_cleaned_personal_property_list.csv",
    "2022": "2022_cleaned_personal_property_list.csv",
    "2023": "2023_cleaned_personal_property_list.csv",
    "2024": "2024_cleaned_personal_property_list.csv",
}

In [ ]:
# Load and merge datasets
dfs = []
for year, path in dataset_paths.items():
    df = pd.read_csv(path)
    df["YEAR"] = int(year)  # Add year column
    dfs.append(df)

In [ ]:
# Combine all data into one DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Convert data types
merged_df["AGENCY_CODE"] = merged_df["AGENCY_CODE"].astype("Int64")
merged_df["PROPERTY_CLASSIFICATION"] = merged_df["PROPERTY_CLASSIFICATION"].astype("category")
merged_df["DATE_ACQUIRED_OR_INSTALLED"] = pd.to_datetime(merged_df["DATE_ACQUIRED_OR_INSTALLED"], errors="coerce")

In [ ]:
# Save the integrated dataset
merged_df.to_csv("merged_cleaned_personal_property_list.csv", index=False)


In [ ]:
# ---- Trends Over the Years: Total Property Value ----
plt.figure(figsize=(10, 5))
merged_df.groupby("YEAR")["VALUE_OR_COST"].sum().plot(marker="o")
plt.xlabel("Year")
plt.ylabel("Total Property Value (USD)")
plt.title("Total Property Value Over the Years")
plt.grid(True)
plt.show()

In [ ]:
# ---- IT vs. Non-IT Property Analysis ----
it_vs_nonit = merged_df.groupby(["YEAR", "IT_OR_NON-IT"]) ["VALUE_OR_COST"].sum().unstack()
plt.figure(figsize=(10, 5))
it_vs_nonit.plot(kind="bar", stacked=True, figsize=(10, 5))
plt.xlabel("Year")
plt.ylabel("Total Value (USD)")
plt.title("IT vs. Non-IT Property Trends Over the Years")
plt.legend(title="Property Type")
plt.grid(axis="y")
plt.show()

In [ ]:
# ---- Acquisition Patterns: Number of Acquisitions per Year ----
plt.figure(figsize=(10, 5))
merged_df.groupby("YEAR")["PROPERTY_NAME"].count().plot(marker="o", color="r")
plt.xlabel("Year")
plt.ylabel("Number of Acquisitions")
plt.title("Number of Property Acquisitions Per Year")
plt.grid(True)
plt.show()

In [20]:
# ---- Predictive Analysis: Property Value Prediction ----
subset_df = merged_df[["YEAR", "AGENCY_CODE", "VALUE_OR_COST"]].dropna()
features = subset_df[["YEAR", "AGENCY_CODE"]]
target = subset_df["VALUE_OR_COST"]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
print("R-Squared Score:", r2_score(y_test, y_pred))

NameError: name 'merged_df' is not defined

In [ ]:
# ---- Clustering Analysis of Agencies based on Acquisitions ----
agency_acquisitions = merged_df.groupby("AGENCY_CODE")["VALUE_OR_COST"].sum().reset_index()
kmeans = KMeans(n_clusters=3, random_state=42)
agency_acquisitions["Cluster"] = kmeans.fit_predict(agency_acquisitions[["VALUE_OR_COST"]])
plt.figure(figsize=(10, 5))
sns.scatterplot(x=agency_acquisitions["AGENCY_CODE"], y=agency_acquisitions["VALUE_OR_COST"], hue=agency_acquisitions["Cluster"], palette="viridis")
plt.xlabel("Agency Code")
plt.ylabel("Total Acquired Property Value")
plt.title("Clustering of Agencies Based on Acquired Property Value")
plt.legend(title="Cluster")
plt.show()


In [ ]:
# Display the first few rows of merged dataset
merged_df.head()